In [2]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import itchat
import os, codecs
import datetime
import threading
import time
from itchat.content import *
from time import strftime, gmtime
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

BotName = "iknow"
# BotName = "里卡多"
chatbot = ChatBot("BotName")

chatbot.set_trainer(ChatterBotCorpusTrainer)
# 使用中文语料库训练它
chatbot.train("chatterbot.corpus.chinese")
# 使用英文语料库训练它
chatbot.train("chatterbot.corpus.english")

conversations.yml Training: [####################] 100%
greetings.yml Training: [####################] 100%
trivia.yml Training: [####################] 100%
ai.yml Training: [####################] 100%
botprofile.yml Training: [####################] 100%
computers.yml Training: [####################] 100%
conversations.yml Training: [####################] 100%
drugs.yml Training: [####################] 100%
emotion.yml Training: [####################] 100%
food.yml Training: [####################] 100%
gossip.yml Training: [####################] 100%
greetings.yml Training: [####################] 100%
history.yml Training: [####################] 100%
humor.yml Training: [####################] 100%
literature.yml Training: [####################] 100%
money.yml Training: [####################] 100%
movies.yml Training: [####################] 100%
politics.yml Training: [####################] 100%
psychology.yml Training: [####################] 100%
science.yml Training: [################

In [11]:
groupID = ''
links = []
FILENAME = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M') + ".txt"
PATH = "./outputweixin/"
if os.path.exists(PATH) == False:
    os.makedirs(PATH)
fp = codecs.open(PATH + FILENAME, "a", "utf-8")

itchat.auto_login(enableCmdQR=False, hotReload=True)

In [12]:
@itchat.msg_register(SHARING, isGroupChat=True)
def text_reply(msg):
    tempLink = {}
    if msg['FromUserName'] == groupID:
        tempLink['time'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        tempLink['text'] = msg.text
        tempLink['ActualNickName'] = msg['ActualNickName']
        tempLink['Url'] = msg['Url']
        links.append(tempLink)
        print(links)

    # itchat.send(msg)
    print(msg['FromUserName'])
    print(msg.text)
    print(msg['Url'])
    print('hahahahhaahhha')

In [13]:
@itchat.msg_register([TEXT, MAP, CARD, NOTE], isFriendChat=True, isGroupChat=True, isMpChat=True)
def text_send_test(msg):
    print('the FromUserName:' + msg['FromUserName'])
    print('the ToUserName:' + msg['ToUserName'])
    res = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + itchat.search_friends(userName=msg['FromUserName'])[
        'NickName'] + " to " + itchat.search_friends(userName=msg['ToUserName'])['NickName'] + ":" + msg['Text']
    fp.write(res + "\n\n")
    print(res)
    res2 = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + itchat.search_friends(userName=msg['ToUserName'])[
        'NickName'] + " to " + itchat.search_friends(userName=msg['FromUserName'])[
               'NickName'] + ":"  # +chatbot.get_response(msg['Text'])
    # fp.write(res2 + "\n\n")
    # print (res2)

    if itchat.search_friends(userName=msg['FromUserName'])['NickName'] != BotName:
        res3 = chatbot.get_response(msg['Text'])
        print('%s%s' % (res2, res3))
        msg.user.send('%s' % (res3))
        fp.write('%s%s\n\n' % (res2, res3))

In [14]:
# @itchat.msg_register([TEXT, MAP, CARD, NOTE, SHARING])
# def text_reply(msg):
#    msg.user.send('%s: %s' % (msg.type, chatbot.get_response(msg['Text'])))

@itchat.msg_register(TEXT, isGroupChat=True)
def Gchat(msg):
    gres = "Group#" + "/" + msg['FromUserName'] + "/" + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + msg[
        'ActualNickName'] + ":" + msg['Text']
    fp.write(gres + "\n\n")
    print(gres)
    if msg.isAt:
        gres2 = "Group#" + "/" + msg['FromUserName'] + "/" + datetime.datetime.now().strftime(
            '%Y-%m-%d %H:%M:%S') + BotName + ":@" + msg['ActualNickName']
        gres3 = chatbot.get_response(msg['Text'])
        msg.user.send(u'@%s\u2005%s' % (
            msg.actualNickName, gres3))
        print('%s%s' % (gres2, gres3))
        fp.write('%s%s\n\n' % (gres2, gres3))

In [15]:
@itchat.msg_register([PICTURE, RECORDING, ATTACHMENT, VIDEO], isFriendChat=True, isGroupChat=True, isMpChat=True)
def download_files(msg):
    IMGPATH = "./images/"
    if os.path.exists(IMGPATH) == False:
        os.makedirs(IMGPATH)
    msg.download(IMGPATH + msg.fileName)
    itchat.send('@%s@%s' % (
        'img' if msg['Type'] == 'Picture' else 'fil', msg['FileName']),
                msg['FromUserName'])
    return '%s received' % msg['Type']

In [16]:
@itchat.msg_register(FRIENDS)
def add_friend(msg):
    msg.user.verify()
    msg.user.send('Nice to meet you!')

In [17]:
def timer(delay):
    '''''
    每n秒执行一次
    '''
    while True:
        # print(time.strftime('%Y-%m-%d %X',time.localtime()))
        #if datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') == '2017-10-29 15:35:00':
        if (datetime.datetime.now().strftime('%H:%M:%S') == '15:35:00') and (links is not []):
            itchat.send('There are some shared links ...', toUserName=linkReceiver)
            for link in links:
                gles = link['time'] + " " + link['ActualNickName'] + ":" + link['text']
                itchat.send(gles, toUserName=linkReceiver)
                itchat.send(link['Url'], toUserName=linkReceiver)
            del links[:]
        time.sleep(delay)

In [ ]:
if __name__ == "__main__":
    try:
        chatrooms = itchat.get_chatrooms(update=True, contactOnly=True)
        chatroom_ids = [c['UserName'] for c in chatrooms]
        groupIDs = [item['UserName'] for item in chatrooms if item['NickName']=='Oversea Master Student Group'] 
        groupID = groupIDs[0]
        print(groupID)
        print(chatrooms)
        print(chatroom_ids)
        print('正在监测的群聊：', len(chatrooms), '个')
        print(' '.join([item['NickName'] for item in chatrooms]))
        threads = []
        #Oversea Master Student Group
        # t = threading.Thread(target = itchat.run,args=())
        # threads.append(t)
        # t.start()
        t = threading.Thread(target=timer, args=(0.1,))
        threads.append(t)
        t.start()
        print('all is ok !!!!!')
        itchat.run()



    except KeyboardInterrupt:
        t.exit()
        fp.close()
        itchat.logout()

Start auto replying.


@@26d2a3d8bcf02b234dad010832d60d91665f2ab22421214fc376c287820012d3
[{'Uin': 0, 'UserName': '@@26d2a3d8bcf02b234dad010832d60d91665f2ab22421214fc376c287820012d3', 'NickName': 'Oversea Master Student Group', 'HeadImgUrl': '/cgi-bin/mmwebwx-bin/webwxgetheadimg?seq=670986072&username=@@26d2a3d8bcf02b234dad010832d60d91665f2ab22421214fc376c287820012d3&skey=@crypt_a537bf3a_ee92df6de838c2c9992c446241c96c3b', 'ContactFlag': 3, 'MemberCount': 0, 'MemberList': [], 'RemarkName': '', 'HideInputBarFlag': 0, 'Sex': 0, 'Signature': '', 'VerifyFlag': 0, 'OwnerUin': 0, 'PYInitial': 'OVERSEAMASTERSTUDENTGROUP', 'PYQuanPin': 'OverseaMasterStudentGroup', 'RemarkPYInitial': '', 'RemarkPYQuanPin': '', 'StarFriend': 0, 'AppAccountFlag': 0, 'Statues': 1, 'AttrStatus': 0, 'Province': '', 'City': '', 'Alias': '', 'SnsFlag': 0, 'UniFriend': 0, 'DisplayName': '', 'ChatRoomId': 0, 'KeyWord': '', 'EncryChatRoomId': '', 'IsOwner': 0}]
['@@26d2a3d8bcf02b234dad010832d60d91665f2ab22421214fc376c287820012d3']
正在监测的群聊： 1 个
